In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib as mpl
import seaborn as sns

import os

import numpy as np
import pandas as pd
import tensorflow as tf

mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80
mpl.rcParams['figure.figsize'] = np.array((10,6))*.6

# my code
import misc
import gan


# Load galaxy image data

In [ ]:
X_img = np.load("data/images.small.npy")
X_img = X_img.transpose([0,2,3,1])
X_img.shape

In [ ]:
image_size = X_img.shape[1]
image_size

# Load targets
`HSC_ids` are in the same order as the `X_img` data. These ids are then used to cross-reference the table read into `df`.

In [ ]:
HSC_ids = np.load("data/HSC_ids.npy")
HSC_ids

In [ ]:
df = pd.read_csv("data/2018_02_23-all_objects.csv")
df = df[df.selected]


df = df.drop_duplicates("HSC_id") \
       .set_index("HSC_id") \
       [["photo_z", "log_mass"]]


df.head()


In [ ]:
y = df.loc[HSC_ids].values
y_for_visualization_samples = np.array([.14, 8.51])

In [ ]:
standardizer = misc.Standardizer(y)
y_standard = standardizer(y)
y_for_visualization_samples_standard = standardizer(y_for_visualization_samples)

y_standard.shape

# Run GAN
Modeled after: https://github.com/hwalsuklee/tensorflow-generative-model-collections/blob/f24a27feba327a1086298a810fdf83bb30d5128a/CGAN.py

In [ ]:
num_threads = 10

sess = tf.Session(config=tf.ConfigProto(
    intra_op_parallelism_threads=num_threads,
    inter_op_parallelism_threads=num_threads,
))

num_epochs = 265
batch_size = 64
z_dim = 100
dataset_name = "galaxy"
checkpoint_dir = os.path.join(os.path.expanduser("~"),
                              "tmp - models",
                              "models/gan/checkpoints")
result_dir = "models/gan/results"
log_dir = "models/gan/log"

model = gan.CGAN(sess, num_epochs, batch_size, z_dim, dataset_name,
                 image_size, X_img, 
                 y_standard, y_for_visualization_samples_standard,
                 checkpoint_dir, result_dir, log_dir,
                 d_learning_rate=.0001,
                 relative_learning_rate=4.,
                )

model.build_model()


In [ ]:
model.train()